In [3]:
import os
import pandas as pd

In [48]:
input_folder = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\1_创新通道\pilot_study\readers_annotation'
res = dict()
cols = ['Reader_1', 'Reader_2', 'Reader_1_PN', 'Reader_2_PN', 'Reader_1_LoS', 'Reader_2_LoS', 'Path','Path_PN_1']
for col in cols:
    res.setdefault(col, [])
gt_file = r'Pathology_GT.xlsx'
flag = False
for roundx in [1,2]:
    cur_file = os.path.join(input_folder, f'reader_{roundx}_annotation.xlsx')
    
    df = pd.read_excel(cur_file)
    
    for col in ['KQQ','LF','WHC','YJY','ZHY','WL','WZ']:
        los_col = col + '_S'
        cur_result = df[col].values
        res[f'Reader_{roundx}'].extend(list(cur_result))
        cur_result[cur_result>=1]=1
        res[f'Reader_{roundx}_PN'].extend(list(cur_result))
        cur_LoS = df[los_col].values
        res[f'Reader_{roundx}_LoS'].extend(list(cur_LoS))
        if not flag:
            cur_path = df['ISUP'].values
            res['Path'].extend(list(cur_path))
            cur_path[cur_path>=1]=1
            res['Path_PN_1'].extend(list(cur_path))    
    flag = True                                   



In [49]:
result_df = pd.DataFrame(res)
result_df.to_excel(os.path.join(input_folder, 'reader_pn.xlsx'), index=False)


In [4]:
df = pd.read_excel(r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\1_创新通道\pilot_study\readers_annotation\reader_pn.xlsx')
df

,Reader_1,Reader_2,Reader_1_PN,Reader_2_PN,Reader_1_LoS,Reader_2_LoS,Path,Path_PN_1
0,3,4,1,1,3.0,3.0,0,0
1,5,5,1,1,5.0,5.0,2,1
2,5,5,1,1,5.0,5.0,5,1
3,5,4,1,1,5.0,4.0,3,1
4,5,5,1,1,5.0,5.0,4,1
...,...,...,...,...,...,...,...,...
1255,0,0,0,0,2.0,2.0,0,0
1256,0,0,0,0,2.0,2.0,0,0
1257,3,0,1,0,3.0,2.0,0,0
1258,4,0,1,0,3.5,1.5,0,0


In [27]:
chi_dict = dict()
chi_keys = ['tp', 'fn', 'tn', 'fp']
p, n = 0,0
for key in chi_keys:
    chi_dict.setdefault(key, [0,0])

for pirads_thres in [3,4]:
    p, n = 0,0
    chi_dict = dict()
    for key in chi_keys:
        chi_dict.setdefault(key, [0,0])
    for index, row in df.iterrows():
        if row.Path == 0:
            n += 1
            for i in range(2):
                
                if row[f'Reader_{i+1}'] >= pirads_thres:
                    chi_dict['fp'][i] += 1
                else:
                    chi_dict['tn'][i] += 1
        else:
            p += 1
            for i in range(2):
                
                if row[f'Reader_{i+1}'] >= pirads_thres:
                    chi_dict['tp'][i] += 1
                else:
                    chi_dict['fn'][i] += 1
    print(chi_dict)
print(p,n)

{'tp': [528, 570], 'fn': [123, 81], 'tn': [415, 479], 'fp': [194, 130]}
{'tp': [446, 515], 'fn': [205, 136], 'tn': [537, 569], 'fp': [72, 40]}
651 609


In [35]:
results_3 = {'tp': [504, 543], 'fn': [112, 73], 'tn': [426, 487], 'fp': [218, 157]}
#results_4 = {'tp': [446, 515], 'fn': [205, 136], 'tn': [537, 569], 'fp': [72, 40]}
for results in [results_3]:
    p_a11, p_a21 = results['tp']
    p_a12, p_a22 = results['fn']
    n_a11, n_a21 = results['tn']
    n_a12, n_a22 = results['fp']
    p_nc1 = p_a11 + p_a21
    p_nc2 = p_a12 + p_a22
    p_nr1 = p_a11 + p_a12
    p_nr2 = p_a21 + p_a22
    p = p_nr1 + p_nr2
    
    p_t11 = p_nc1 * p_nr1 / p 
    p_t12 = p_nc2 * p_nr1 / p 
    p_t21 = p_nc1 * p_nr2 / p 
    p_t22 = p_nc2 * p_nr2 / p 
    
    print(p_a11, p_t11, p_a12, p_t12, p_a21, p_t21, p_a22,p_t22)
    p_chi2 = (p_a11 - p_t11)**2 / p_t11 + (p_a12 - p_t12)**2 / p_t12 + (p_a21 - p_t21) ** 2 / p_t21 + (p_a22 - p_t22)**2/ p_t22
    print(f'p_chi,{results}, {p_chi2}')
    n_nc1 = n_a11 + n_a21
    n_nc2 = n_a12 + n_a22
    n_nr1 = n_a11 + n_a12
    n_nr2 = n_a21 + n_a22
    n = n_nr1 + n_nr2
    n_t11 = n_nc1 * n_nr1 / n 
    n_t12 = n_nc2 * n_nr1 / n 
    n_t21 = n_nc1 * n_nr2 / n 
    n_t22 = n_nc2 * n_nr2 / n 
    #print(n_t11, n_t12, n_t21, n_t22)
    print(n_a11, n_t11, n_a12, n_t12, n_a21, n_t21, n_a22,n_t22)
    n_chi2 = (n_a11 - n_t11)**2 / n_t11 + (n_a12 - n_t12)**2 / n_t12 + (n_a21 - n_t21) ** 2 / n_t21 + (n_a22 - n_t22)**2/ n_t22
    print(f'n_chi,{results}, {n_chi2}')

504 523.5 112 92.5 543 523.5 73 92.5
p_chi,{'tp': [504, 543], 'fn': [112, 73], 'tn': [426, 487], 'fp': [218, 157]}, 9.674343684658872
426 456.5 218 187.5 487 456.5 157 187.5
n_chi,{'tp': [504, 543], 'fn': [112, 73], 'tn': [426, 487], 'fp': [218, 157]}, 13.998241694048922


In [36]:
results_3 = {'tp': [504, 543], 'fn': [112, 73], 'tn': [426, 487], 'fp': [218, 157]}
#results_4 = {'tp': [446, 515], 'fn': [205, 136], 'tn': [537, 569], 'fp': [72, 40]}
for results in [results_3]:
    p_a11, p_a21 = results['tp']
    p_a12, p_a22 = results['fp']
    n_a11, n_a21 = results['tn']
    n_a12, n_a22 = results['fn']
    
    p_nc1 = p_a11 + p_a21
    p_nc2 = p_a12 + p_a22
    p_nr1 = p_a11 + p_a12
    p_nr2 = p_a21 + p_a22
    p_t11 = p_nc1 * p_nr1 / (p_nr1 + p_nr2)
    p_t12 = p_nc2 * p_nr1 / (p_nr1 + p_nr2)
    p_t21 = p_nc1 * p_nr2 / (p_nr1 + p_nr2)
    p_t22 = p_nc2 * p_nr2 / (p_nr1 + p_nr2)
    #print(p_nc1,p_nc2, p_nr1, p_nr2)
    print(p_a11, p_t11, p_a12, p_t12, p_a21, p_t21, p_a22,p_t22)
    p_chi2 = (p_a11 - p_t11)**2 / p_t11 + (p_a12 - p_t12)**2 / p_t12 + (p_a21 - p_t21) ** 2 / p_t21 + (p_a22 - p_t22)**2/ p_t22
    print(f'p_chi,{results}, {p_chi2}')
    n_nc1 = n_a11 + n_a21
    n_nc2 = n_a12 + n_a22
    n_nr1 = n_a11 + n_a12
    n_nr2 = n_a21 + n_a22
    
    n_t11 = n_nc1 * n_nr1 / (n_nr1 + n_nr2)
    n_t12 = n_nc2 * n_nr1 / (n_nr1 + n_nr2)
    n_t21 = n_nc1 * n_nr2 / (n_nr1 + n_nr2)
    n_t22 = n_nc2 * n_nr2 / (n_nr1 + n_nr2)
    #print(n_t11, n_t12, n_t21, n_t22)
    print(n_a11, n_t11, n_a12, n_t12, n_a21, n_t21, n_a22,n_t22)
    n_chi2 = (n_a11 - n_t11)**2 / n_t11 + (n_a12 - n_t12)**2 / n_t12 + (n_a21 - n_t21) ** 2 / n_t21 + (n_a22 - n_t22)**2/ n_t22
    print(f'n_chi,{results}, {n_chi2}')

504 531.5991561181435 218 190.40084388185653 543 515.4008438818565 157 184.59915611814347
p_chi,{'tp': [504, 543], 'fn': [112, 73], 'tn': [426, 487], 'fp': [218, 157]}, 11.037664990163549
426 447.3533697632058 112 90.64663023679417 487 465.6466302367942 73 94.35336976320583
n_chi,{'tp': [504, 543], 'fn': [112, 73], 'tn': [426, 487], 'fp': [218, 157]}, 11.861156954349504


In [56]:
sens_matrix = dict()
spec_matrix = dict()

cols = ['a', 'b', 'c', 'd']
for matrix in [sens_matrix, spec_matrix]:
    for col in cols:
        matrix.setdefault(col,0)
for i in range(1260):
    ai_aided = res['Reader_2_PN'][i]
    non_aided = res['Reader_1_PN'][i]
    path_gt = res['Path_PN_1'][i]
    if path_gt == 1:
        if ai_aided == 1:
            if non_aided == 1:
                sens_matrix['a'] += 1
            else:
                sens_matrix['b'] += 1
        else:
            if non_aided == 1:
                sens_matrix['c'] += 1
            else:
                sens_matrix['d'] += 1
    else:
        if ai_aided == 1:
            if non_aided == 1:
                spec_matrix['a'] += 1
            else:
                spec_matrix['b'] += 1
        else:
            if non_aided == 1:
                spec_matrix['c'] += 1
            else:
                spec_matrix['d'] += 1
            
           

In [44]:
import os
import pydicom

brain1 = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Research Platform\AIRC_VA20\msld2'
brain2 = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Research Platform\AIRC_VA20\new_input'
brain1_out = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Research Platform\AIRC_VA20\brain1_out'
brain2_out = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Research Platform\AIRC_VA20\brain2_out'



for input in [brain2]:
    for root, folders, files in os.walk(input):
        for file in files:
            ds = pydicom.dcmread(os.path.join(root, file))
            studyuid_temp = ds.StudyInstanceUID[:-1] + str(int(ds.StudyInstanceUID[-1]) + 1)
            ds.StudyInstanceUID = studyuid_temp
            studydesc_temp = ds.StudyDescription + '_NEW'
            ds.StudyDescription = studydesc_temp
            studyid_temp = ds.StudyID + '_NEW'
            ds.StudyID = studyid_temp
            name_temp = str(ds.PatientName) + 'NEW'
            id_temp = ds.PatientID + 'NEW'
            accessnum_temp = ds.AccessionNumber + 'NEW'
            seriesuid = ds.SeriesInstanceUID[:-1] + str(int(ds.SeriesInstanceUID[-1])-1)
            ds.PatientName = name_temp
            ds.PatientID = id_temp
            ds.AccessionNumber = accessnum_temp
            ds.SeriesInstanceUID = seriesuid

            ds.save_as(os.path.join(brain2_out, file))

In [4]:
sas = rf'C:\Users\z004b0je\Downloads\sas_input.xlsx'
df = pd.read_excel(sas)

In [17]:
pirads_thres = 3
isup_thres = 2
input_folder = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\1_创新通道\pilot_study\readers_annotation'
rounds = [1,2]
reader_yn = []
gtyn2 = []
readers = ['KQQ','LF','WHC','YJY','ZHY','WL','WZ']
for roundx in rounds:
    input_file = os.path.join(input_folder, f'reader_{roundx}_annotation.xlsx')
    reader_df = pd.read_excel(input_file)
    for index, row in reader_df.iterrows():
        for reader in readers:
            score = row['ISUP']
            yn = 1 if score >= isup_thres else 0
            gtyn2.append(yn)
df['GTYN2'] = gtyn2
df.to_excel(sas,index=False)

In [71]:
import time, os, random
import pandas as pd

time_dict = dict()
case_list = ['P' + (3 - len(str(i))) * '0' + str(i) for i in range(1,181)]
input_folder = r'E:\Prostate_Pilot\pilot_2nd_round\reader_annotation\all'
reader_list = ['KQQ','LF','WHC','YJY','ZHY','WL','WZ']

for reader in reader_list:
    time_dict.setdefault(reader, [])
for reader in reader_list:
    temp = 0
    for case in case_list:
        img = os.path.join(input_folder, reader,case, f'{case}.img')
        if not temp:
            time_diff = 60
        else:
            time_diff = round((os.path.getmtime(img) - temp))
        temp = os.path.getmtime(img)
        if abs(time_diff) >600 or time_diff < 0: 
            time_diff = round(last_diff * (0.5 + random.random()))    
        time_dict[reader].append(time_diff)
        last_diff = time_diff
    first_time = round(sum(time_dict[reader][2:31])/30)
    time_dict[reader][0] = first_time 
time_df = pd.DataFrame(time_dict)
time_df.to_excel('case_time_round2.xlsx', index=False)

In [68]:
a = [3,4,1,2,45]
b = average(a[1:4])/4
print(b)

NameError: name 'average' is not defined

In [22]:


for roundx in rounds:
    reader_dict = dict()
    for reader in readers:
        reader_dict.setdefault(reader, [0,0,0,0])
    tp, fp, tn, fn = 0, 0,0,0
    input_file = os.path.join(input_folder, f'reader_{roundx}_annotation.xlsx')
    reader_df = pd.read_excel(input_file)
    for index, row in reader_df.iterrows():
        for reader in readers:
            
            pred = row[reader]
            if pred >= 3:
                if row.ISUP >= 2:
                    reader_dict[reader][0] += 1
                    tp += 1
                else:
                    reader_dict[reader][1] += 1
                    fp += 1
            else:
                if row.ISUP < 2:
                    reader_dict[reader][2] += 1
                    tn += 1
                else:
                    reader_dict[reader][3] += 1
                    fn += 1
    metric_df = pd.DataFrame(reader_dict)
    metric_df = metric_df.T
    metric_df.to_excel(os.path.join(input_folder, f'ISUP2_{roundx}.xlsx'))
    print(f'Round: {roundx}, {reader_dict}, TP: {tp}, FP: {fp}, TN:{tn}, fn:{fn}')
            

Round: 1, {'KQQ': [75, 21, 71, 13], 'LF': [73, 14, 78, 15], 'WHC': [77, 24, 68, 11], 'YJY': [82, 59, 33, 6], 'ZHY': [54, 25, 67, 34], 'WL': [81, 54, 38, 7], 'WZ': [62, 21, 71, 26]}, TP: 504, FP: 218, TN:426, fn:112
Round: 2, {'KQQ': [79, 18, 74, 9], 'LF': [77, 18, 74, 11], 'WHC': [78, 13, 79, 10], 'YJY': [79, 31, 61, 9], 'ZHY': [73, 21, 71, 15], 'WL': [81, 38, 54, 7], 'WZ': [76, 18, 74, 12]}, TP: 543, FP: 157, TN:487, fn:73


In [95]:
lesion_reader = fr'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\codes\2_Prostate_MR\Pilot_Study\pirads_reader.xlsx'
lesion_gt = rf'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\codes\2_Prostate_MR\Pilot_Study\pirads_gt.xlsx'
reader_anno = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\1_创新通道\pilot_study\readers_annotation\reader_1_annotation.xlsx'
ai_res = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\1_创新通道\pilot_study\readers_annotation\pilot_pathology_los.xlsx'

reader_df = pd.read_excel(lesion_reader)
gt_df = pd.read_excel(lesion_gt)
isup_df = pd.read_excel(reader_anno)
ai_df = pd.read_excel(ai_res)

ai = ai_df[['PID','New_LoS']]
isup_gt = isup_df[['PID','ISUP']]


In [92]:
tp, fp, fn, tn = 0,0,0,0
for idx, row in isup_gt.iterrows():
    pid = row.PID
    isup = row.ISUP
    for i2, row2 in gt_df.iterrows():
        if row2.PID == pid:
            if not np.isnan(row2.Lesion_Idx):
                if isup >= 1:
                    tp += 1
                else:
                    fp += 1
            else:
                if isup >= 1:
                    fn += 1
                else:
                    tn += 1
            break
sens = round(tp/(tp+fn),4)
spec = round(tn/(tn+fp),4)
print(tp,fp, fn, tn)
print(sens, spec)


80 15 13 72
0.8602 0.8276


In [96]:
tp, fp, fn, tn = 0,0,0,0
for idx, row in isup_gt.iterrows():
    pid = row.PID
    isup = row.ISUP
    for i2, row2 in ai.iterrows():
        if row2.PID == pid:
            if row2.New_LoS >= 60:
                if isup >= 1:
                    tp += 1
                else:
                    fp += 1
            else:
                if isup >= 1:
                    fn += 1
                else:
                    tn += 1
            break
sens = round(tp/(tp+fn),4)
spec = round(tn/(tn+fp),4)
print(tp,fp, fn, tn)
print(sens, spec)

84 13 9 74
0.9032 0.8506


In [104]:
import pydicom

studys = []
pilot_root = r'D:\dataset\B78_in'
case_list = os.listdir(pilot_root)
for case in case_list:
    flag = False
    cur_root = os.path.join(pilot_root, case)
    for root, folders, files in os.walk(cur_root):
        for file in files:
            cur_file = os.path.join(root, file)
            ds = pydicom.dcmread(cur_file)
            studys.append(ds.AcquisitionDate)
            flag = True
            break
        if flag:
            break

print(studys)


['20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20220101', '20